<img src="https://preview.ibb.co/iDivJc/5529.png" width='800' >

# Data: House Sales in King County, USA

### https://www.kaggle.com/harlfoxem/housesalesprediction

In [2]:
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'fH7tipNa2VfJt4LO42GJU2-f65WmKRa0ZYn_H3sUnLU9',
    'service_id': 'iam-ServiceId-5cf22436-1ea4-4a82-8752-22c7e7206d58',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_b0d31744b6c54747aec215514fff27a4_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('kc_house_data.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
df.take(1)

[Row(id='7129300520', date='20141013T000000', price='221900', bedrooms='3', bathrooms='1', sqft_living='1180', sqft_lot='5650', floors='1', waterfront='0', view='0', condition='3', grade='7', sqft_above='1180', sqft_basement='0', yr_built='1955', yr_renovated='0', zipcode='98178', lat='47.5112', long='-122.257', sqft_living15='1340', sqft_lot15='5650')]

# Clean your ... data

In [3]:
df.show()

+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|     price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|    221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|    538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|       

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sqft_above: string (nullable = true)
 |-- sqft_basement: string (nullable = true)
 |-- yr_built: string (nullable = true)
 |-- yr_renovated: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- sqft_living15: string (nullable = true)
 |-- sqft_lot15: string (nullable = true)



<img src="https://i.imgflip.com/294a8s.jpg" />

In [5]:
from pyspark.sql.functions import udf  # user defined function
parse_date = udf(lambda date: date.split("T")[0])
df = df.withColumn("date", parse_date(df.date))

In [6]:
for column in df.columns:
    numeric_column = df[column].cast('float')
    df = df.withColumn(column, numeric_column)
df.printSchema()

root
 |-- id: float (nullable = true)
 |-- date: float (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: float (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: float (nullable = true)
 |-- waterfront: float (nullable = true)
 |-- view: float (nullable = true)
 |-- condition: float (nullable = true)
 |-- grade: float (nullable = true)
 |-- sqft_above: float (nullable = true)
 |-- sqft_basement: float (nullable = true)
 |-- yr_built: float (nullable = true)
 |-- yr_renovated: float (nullable = true)
 |-- zipcode: float (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- sqft_living15: float (nullable = true)
 |-- sqft_lot15: float (nullable = true)



In [7]:
df.show(5)

+------------+-----------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|          id|       date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+------------+-----------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
| 7.1293005E9|2.0141012E7|221900.0|     3.0|      1.0|     1180.0|  5650.0|   1.0|       0.0| 0.0|      3.0|  7.0|    1180.0|          0.0|  1955.0|         0.0|98178.0|47.5112|-122.257|       1340.0|    5650.0|
|    6.4141E9|2.0141208E7|538000.0|     3.0|     2.25|     2570.0|  7242.0|   2.0|       0.0| 0.0|      3.0|  7.0|    2170.0|        400.0|  1951.0|    

# Transformers
## VectorAssembler is a transformer that combines a given list of columns into a single vector column.

In [34]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
import numpy as np

In [23]:
features = list()
for col in df.columns:
    if 'price' in col or 'id' in col:continue
    features.append(col)

In [37]:
assembler = VectorAssembler(
    inputCols=features,
    outputCol="features")

processed_df = assembler.transform(df).select(["price","features"])
processed_df.show(1)

+--------+--------------------+
|   price|            features|
+--------+--------------------+
|221900.0|[2.0141012E7,3.0,...|
+--------+--------------------+
only showing top 1 row



In [41]:
train, test = processed_df.randomSplit([0.8, 0.2], seed=42)  # train test split
train.count(),test.count()

(17315, 4298)

# Can we predict the price of a house based on the other features?
# create
# fit
# transform vs predict

In [42]:
from pyspark.ml.feature import StandardScaler

In [43]:
standard_scaler = StandardScaler(inputCol="features", outputCol="scaled", withMean=True, withStd=True)

scaler_model = standard_scaler.fit(train)

train_scaled = scaler_model.transform(train)
test_scaled = scaler_model.transform(test)

train_scaled.head()

Row(price=75000.0, features=DenseVector([20150216.0, 1.0, 0.0, 670.0, 43377.0, 1.0, 0.0, 0.0, 3.0, 3.0, 670.0, 0.0, 1966.0, 0.0, 98022.0, 47.2638, -121.906, 1160.0, 42882.0]), scaled=DenseVector([1.4183, -2.5312, -2.7476, -1.5342, 0.6539, -0.9149, -0.087, -0.3064, -0.6297, -3.962, -1.3494, -0.6567, -0.1664, -0.2116, -1.0452, -2.1319, 2.1826, -1.2064, 1.0891]))

In [56]:
train_scaled.show(1)

+-------+--------------------+--------------------+
|  price|            features|              scaled|
+-------+--------------------+--------------------+
|75000.0|[2.0150216E7,1.0,...|[1.41829140513976...|
+-------+--------------------+--------------------+
only showing top 1 row



In [61]:
from pyspark.ml.regression import LinearRegression

In [74]:
lr = LinearRegression(featuresCol='scaled', labelCol='price') # create

In [75]:
model = lr.fit(train_scaled) # fit

In [79]:
model.coefficients

DenseVector([13606.0926, -31790.3253, 30115.8155, 1488517.6311, 6365.1685, 5481.8402, 50738.6515, 37825.4152, 18168.775, 110191.1272, -1197338.7843, -652990.4813, -75948.2601, 10039.1401, -31536.0726, 84295.9861, -29543.2436, 18403.0739, -11026.5998])

In [81]:
preds = model.transform(test_scaled)
preds.show(1)

+-------+--------------------+--------------------+-------------------+
|  price|            features|              scaled|         prediction|
+-------+--------------------+--------------------+-------------------+
|78000.0|[2.0140506E7,2.0,...|[-0.7679014231535...|-51504.086829517735|
+-------+--------------------+--------------------+-------------------+
only showing top 1 row



In [82]:
from pyspark.ml.evaluation import RegressionEvaluator

In [84]:
re = RegressionEvaluator(predictionCol='prediction', labelCol='price', metricName='rmse')

In [86]:
re.evaluate(preds)

214653.21993681777